In [5]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pytesseract
from PIL import Image, ImageFilter
import cv2
from collections import Counter
import os
import time
import re
from datetime import datetime, timedelta
import pandas as pd
import csv
import ast
import pyodbc
import json
import concurrent.futures

In [3]:
with open("account.csv", "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    account_list = [row for row in reader]
for account in account_list[:]:
    print("Account: ",account['username'],account['password'])

Account:  3603294570 SEOGWANG2020**
Account:  3603312212 3603312212
Account:  1102022318 Bongsanvina
Account:  2300956022 2300956022
Account:  2400859625 Sjvietnam
Account:  3502391144 3502391144
Account:  3702789197 3702789197
Account:  sedovina sedovina


In [6]:
def generate_dates(start_date, end_date):
    periods = []
    current_end = datetime.strptime(end_date, "%d/%m/%Y")
    current_start = current_end - timedelta(days=10)
    
    while current_start >= datetime.strptime(start_date, "%d/%m/%Y"):
        periods.append((current_start.strftime("%d/%m/%Y"), current_end.strftime("%d/%m/%Y")))
        # Lùi lại một tuần
        current_end = current_start - timedelta(days=1)
        current_start = current_end - timedelta(days=10)
    periods.append((start_date, current_end.strftime("%d/%m/%Y")))

    return periods

# Ngày bắt đầu là ngày hiện tại
start_date = (datetime.now() - timedelta(days=180)).strftime("%d/%m/%Y")
# Ngày kết thúc là ngày hiện tại + 1 ngày
end_date = (datetime.now() + timedelta(days=1)).strftime("%d/%m/%Y")

# Tạo danh sách các khoảng thời gian
dates_list = generate_dates(start_date, end_date)

len(dates_list)

17

In [7]:
dates_list

[('25/12/2024', '04/01/2025'),
 ('14/12/2024', '24/12/2024'),
 ('03/12/2024', '13/12/2024'),
 ('22/11/2024', '02/12/2024'),
 ('11/11/2024', '21/11/2024'),
 ('31/10/2024', '10/11/2024'),
 ('20/10/2024', '30/10/2024'),
 ('09/10/2024', '19/10/2024'),
 ('28/09/2024', '08/10/2024'),
 ('17/09/2024', '27/09/2024'),
 ('06/09/2024', '16/09/2024'),
 ('26/08/2024', '05/09/2024'),
 ('15/08/2024', '25/08/2024'),
 ('04/08/2024', '14/08/2024'),
 ('24/07/2024', '03/08/2024'),
 ('13/07/2024', '23/07/2024'),
 ('07/07/2024', '12/07/2024')]